In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras import optimizers
from keras import backend

In [4]:
data = pd.read_csv("simulate-circle.csv",header=0)
# data = pd.read_csv("simulate-police.csv",header=0)

In [5]:
data = data.drop(['x11','x12','x3','x4','x5','x6','x7','x8','x9','x10'], axis=1)

In [6]:
data.head()

,x1,x2,y
0,2.151302,1.261139,1
1,-0.115938,1.591839,0
2,1.108706,-0.060229,1
3,-2.637907,-0.257359,1
4,-0.104738,1.995355,0


In [9]:
xtrain, xtest, ytrain, ytest = train_test_split(data.drop('y', axis=1), data["y"], test_size=0.2, random_state=42)
split = int(len(ytest)/2)
xvalid = xtest[:split]
yvalid = ytest[:split]
xtest = xtest[split:]
ytest = ytest[split:]

In [12]:
# def rmse(y_true, y_pred):
#     return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

In [19]:
model = Sequential()
model.add(Dense(30, activation="tanh", input_shape=(xtrain.shape[1],)))
model.add(Dropout(0.1))
model.add(Dense(30, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(30, activation="relu"))
# model.add(Dropout(0.1))
# model.add(Dense(30, activation="relu"))
model.add(Dense(2, activation="softmax"))
model.compile(loss='mean_squared_error', optimizer=optimizers.Adam(lr=0.002), metrics=[rmse])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 30)                90        
_________________________________________________________________
dropout_7 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 30)                930       
_________________________________________________________________
dropout_8 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 30)                930       
_________________________________________________________________
dense_16 (Dense)             (None, 2)                 62        
Total params: 2,012
Trainable params: 2,012
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
keras_callbacks = [
    # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', save_best_only=True, verbose=2)
    # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}.hdf5', monitor='val_loss', save_best_only=True, verbose=0)
    # TensorBoard(log_dir='/tmp/keras_logs/model_3', histogram_freq=0, write_graph=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None),
    EarlyStopping(monitor='val_loss', patience=30, verbose=0)
]

In [ ]:
hist = model.fit(xtrain, ytrain, epochs=200, batch_size=2048, verbose=1, callbacks=keras_callbacks, validation_data=(xvalid, yvalid))

In [21]:
train_score = model.evaluate(xtrain, ytrain, verbose=1)
valid_score = model.evaluate(xvalid, yvalid, verbose=1)

print('Train RMSE: ', round(train_score[1], 4)) 
print('Val RMSE: ', round(valid_score[1], 4))

ValueError: Error when checking target: expected dense_16 to have shape (None, 2) but got array with shape (8000, 1)